In [1]:
# Data
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import impl.layer as l

# Dataset preparation and pre-processing
mnist = input_data.read_data_sets('data/MNIST_data/', one_hot=False)

X_train, y_train = mnist.train.images, mnist.train.labels
X_val, y_val = mnist.validation.images, mnist.validation.labels
X_test, y_test = mnist.test.images, mnist.test.labels
y_test.shape, y_val.shape, y_train.shape

Extracting data/MNIST_data/train-images-idx3-ubyte.gz
Extracting data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting data/MNIST_data/t10k-labels-idx1-ubyte.gz


((10000,), (5000,), (55000,))

In [2]:
def selu_forward(X):
    alpha = 1.6732632423543772848170429916717
    scale = 1.0507009873554804934193349852946
    out = scale * np.where(X>=0.0, X, alpha * (np.exp(X)-1))
    cache = X
    return out, cache

def selu_backward(dout, cache):
    alpha = 1.6732632423543772848170429916717
    scale = 1.0507009873554804934193349852946
    X = cache
    dX_pos = dout.copy()
    dX_pos[X<0] = 0
    dX_neg = dout.copy()
    dX_neg[X>0] = 0
    dX = scale * np.where(X>=0.0, dX_pos, dX_neg * alpha * np.exp(X))
    return dX

# Pre-processing
def prepro(X_train, X_val, X_test):
    mean = np.mean(X_train)
    # scale = 255. - mean # std or sqrt(var), 255 == 2**8 or 8 bit grayscale
    # return (X_train - mean)/ scale, (X_val - mean)/ scale, (X_test - mean) / scale
    return X_train - mean, X_val - mean, X_test - mean

X_train, X_val, X_test = prepro(X_train, X_val, X_test)

In [4]:
# Model
import impl.layer as l # or from impl.layer import *
from impl.loss import * # import all functions from impl.loss file # import impl.loss as loss_func
from sklearn.utils import shuffle as skshuffle

class FFNN:

    def __init__(self, D, C, H, L):
        # self.mode = 'classification'
        self.L = L # number of layers or depth
        self.losses = {'train':[], 'smooth train':[], 'valid':[], 'valid_acc':[]}
        
        # The adaptive/learnable/updatable random feedforward
        self.model = []
        self.W_fixed = []
        low, high = -1, 1
        
        # Input layer
        #         m = dict(W=np.random.randn(D, H) / np.sqrt(D / 2.),
        #                  b=np.zeros((1, H)))
        m = dict(W=np.random.uniform(size=(D, H), low=low, high=high) / np.sqrt(D / 2.),
                 b=np.zeros((1, H)))
        self.model.append(m)
        # Fixed feedback weight
        #         m = np.random.randn(D, H) / np.sqrt(D / 2.)
        m = np.random.uniform(size=(D, H), low=low, high=high) / np.sqrt(D / 2.)
        self.W_fixed.append(m)

        # Hidden layers
        #         m = dict(W=np.random.randn(H, H) / np.sqrt(H / 2.),
        #                  b=np.zeros((1, H)))
        m_L = []
        for _ in range(L):
            m = dict(W=np.random.uniform(size=(H, H), low=low, high=high) / np.sqrt(H / 2.),
                     b=np.zeros((1, H)))
            m_L.append(m)
        self.model.append(m_L)
        # Fixed feedback weight
        #         m = np.random.randn(H, H) / np.sqrt(H / 2.)
        m_L = []
        for _ in range(L):
            m = np.random.uniform(size=(H, H), low=low, high=high) / np.sqrt(H / 2.)
            m_L.append(m)
        self.W_fixed.append(m_L)
        
        # Output layer
        #         m = dict(W=np.random.randn(H, C) / np.sqrt(H / 2.),
        #                  b=np.zeros((1, C)))
        m = dict(W=np.random.uniform(size=(H, C), low=low, high=high) / np.sqrt(H / 2.),
                 b=np.zeros((1, C)))
        self.model.append(m)
        # Fixed feedback weight
        m = np.random.uniform(size=(H, C), low=low, high=high) / np.sqrt(H / 2.)
        #         m = np.random.randn(H, C) / np.sqrt(H / 2.)
        self.W_fixed.append(m)
        
    def fc_forward(self, X, W, b):
        out = (X @ W) + b
        cache = (W, X)
        return out, cache

    def fc_backward(self, dout, cache, W_fixed):
        W, X = cache

        dW = X.T @ dout
        db = np.sum(dout, axis=0).reshape(1, -1) # db_1xn
        # dX = dout @ W.T # Backprop
        dX = dout @ W_fixed.T # fb alignment

        return dX, dW, db

    def train_forward(self, X):
        caches = []
        
        # Input layer
        y, fc_cache = self.fc_forward(X=X, W=self.model[0]['W'], b=self.model[0]['b']) # X_1xD, y_1xc
        y, nl_cache = selu_forward(X=y)
        X = y.copy() # pass the previous output to the next layer
        caches.append((fc_cache, nl_cache)) # caches[0]
        
        # Hidden layers
        fc_caches, nl_caches = [], []
        for layer in range(self.L):
            y, fc_cache = self.fc_forward(X=X, W=self.model[1][layer]['W'], b=self.model[1][layer]['b'])
            y, nl_cache = selu_forward(X=y)
            X = y.copy() # pass to next layer
            fc_caches.append(fc_cache)
            nl_caches.append(nl_cache)
        caches.append((fc_caches, nl_caches)) # caches[1]
        
        # Output layer
        y, fc_cache = self.fc_forward(X=X, W=self.model[2]['W'], b=self.model[2]['b'])
        caches.append(fc_cache) # caches[2]

        return y, caches

    def loss_function(self, y, y_train):
        
        loss = cross_entropy(y, y_train) # softmax is included
        dy = dcross_entropy(y, y_train) # dsoftmax is included
        
        return loss, dy
    
    def train_backward(self, dy, caches):
        grads = []

        # Input layer
        grads.append({key: np.zeros_like(val) for key, val in self.model[0].items()})
        
        # Hidden layer
        grad = []
        for layer in range(self.L):
            grad.append({key: np.zeros_like(val) for key, val in self.model[1][layer].items()})
        grads.append(grad)

        # Outout layer
        grads.append({key: np.zeros_like(val) for key, val in self.model[2].items()})

        # Output layer
        fc_cache = caches[2]
        dX, dW, db = self.fc_backward(dout=dy, cache=fc_cache, W_fixed=self.W_fixed[2])
        dy = dX.copy() # pass it to the previous layer
        grads[2]['W'] = dW
        grads[2]['b'] = db

        # Hidden layer
        fc_caches, nl_caches = caches[1]
        for layer in reversed(range(self.L)):
#             dy = selu_backward(dout=dy, cache=nl_caches[layer])
            dX, dW, db = self.fc_backward(dout=dy, cache=fc_caches[layer], W_fixed=self.W_fixed[1][layer])
            dy = dX.copy()
            grads[1][layer]['W'] = dW
            grads[1][layer]['b'] = db
        
        # Input layer
        fc_cache, nl_cache = caches[0]
#         dy = selu_backward(dout=dy, cache=nl_cache)
        dX, dW, db = self.fc_backward(dout=dy, cache=fc_cache, W_fixed=self.W_fixed[0])
        grads[0]['W'] = dW
        grads[0]['b'] = db

        return dX, grads
    
    def test(self, X):
        y_logit, _ = self.train_forward(X)
        
        # if self.mode == 'classification':
        y_prob = l.softmax(y_logit) # for accuracy== acc
        y_pred = np.argmax(y_prob, axis=1) # for loss==err
        
        return y_pred, y_logit
        
    def get_minibatch(self, X, y, minibatch_size, shuffle):
        minibatches = []

        if shuffle:
            X, y = skshuffle(X, y)

        for i in range(0, X.shape[0], minibatch_size):
            X_mini = X[i:i + minibatch_size]
            y_mini = y[i:i + minibatch_size]
            minibatches.append((X_mini, y_mini))

        return minibatches

    def adam(self, train_set, val_set, alpha, mb_size, n_iter, print_after):
        X_train, y_train = train_set
        X_val, y_val = val_set

        # Momentums
        M, R = [], []

        # Input layer momentums
        M.append({key: np.zeros_like(val) for key, val in self.model[0].items()})
        R.append({key: np.zeros_like(val) for key, val in self.model[0].items()})

        # Hidden layers momentum
        M_, R_ = [], []
        for layer in range(self.L):
            M_.append({key: np.zeros_like(val) for key, val in self.model[1][layer].items()})
            R_.append({key: np.zeros_like(val) for key, val in self.model[1][layer].items()})
        M.append(M_)
        R.append(R_)

        # Output layer momentums
        M.append({key: np.zeros_like(val) for key, val in self.model[2].items()})
        R.append({key: np.zeros_like(val) for key, val in self.model[2].items()})
 
        # Learning decay
        beta1 = .9
        beta2 = .99
        smooth_train = 1.

        # Epochs
        for iter in range(1, n_iter + 1):

            # Minibatches
            #         """
            #         Single training step over minibatch: forward, loss, backprop
            #         """
            # Shuffle for each epochs/ stochasticity/ randomly choosing
            #             for idx in range(len(minibatches)):
            #             for _ in range(10):
            # Shuffle in every iteration
            # The dataset is static and non-sequentiol: no time-dependency or temporal pattern
            minibatches = self.get_minibatch(X_train, y_train, mb_size, shuffle=True)
            idx = np.random.randint(0, len(minibatches))
            X_mini, y_mini = minibatches[idx]
            
            # Train the model
            y, caches = self.train_forward(X_mini)
            loss, dy = self.loss_function(y, y_mini)
            _, grads = self.train_backward(dy, caches)
            self.losses['train'].append(loss)
            smooth_train = (0.999 * smooth_train) + (0.001 * loss)
            self.losses['smooth train'].append(smooth_train)
            
            # Update the model for input layer
            for key in grads[0].keys():
                self.model[0][key] -= alpha * grads[0][key]
                # M[0][key] = l.exp_running_avg(M[0][key], grads[0][key], beta1)
                # R[0][key] = l.exp_running_avg(R[0][key], grads[0][key]**2, beta2)
                # m_k_hat = M[0][key] / (1. - (beta1**(iter)))
                # r_k_hat = R[0][key] / (1. - (beta2**(iter)))
                # self.model[0][key] -= alpha * m_k_hat / (np.sqrt(r_k_hat) + l.eps)

            # Update the model for the hidden layers
            for layer in range(self.L):
                for key in grads[1][layer].keys():
                    self.model[1][layer][key] -= alpha * grads[1][layer][key]
                    # M[1][layer][key] = l.exp_running_avg(M[1][layer][key], grads[1][layer][key], beta1)
                    # R[1][layer][key] = l.exp_running_avg(R[1][layer][key], grads[1][layer][key]**2, beta2)
                    # m_k_hat = M[1][layer][key] / (1. - (beta1**(iter)))
                    # r_k_hat = R[1][layer][key] / (1. - (beta2**(iter)))
                    # self.model[1][layer][key] -= alpha * m_k_hat / (np.sqrt(r_k_hat) + l.eps)


            # Update the model for output layer
            for key in grads[2].keys():
                self.model[2][key] -= alpha * grads[2][key]
                # M[2][key] = l.exp_running_avg(M[2][key], grads[2][key], beta1)
                # R[2][key] = l.exp_running_avg(R[2][key], grads[2][key]**2, beta2)
                # m_k_hat = M[2][key] / (1. - (beta1**(iter)))
                # r_k_hat = R[2][key] / (1. - (beta2**(iter)))
                # self.model[2][key] -= alpha * m_k_hat / (np.sqrt(r_k_hat) + l.eps)

            # Validate the updated model
            y_pred, y_logit = self.test(X_val)
            valid_loss, _ = self.loss_function(y_logit, y_val) # softmax is included in entropy loss function
            self.losses['valid'].append(valid_loss)
            valid_acc = np.mean(y_pred == y_val)
            self.losses['valid_acc'].append(valid_acc)
            
            # Print the model info: loss & accuracy or err & acc
            if iter % print_after == 0:
                print('Iter-{} train loss: {:.4f} valid loss: {:.4f}, valid accuracy: {:.4f}'.format(
                    iter, loss, valid_loss, valid_acc))

        # Test the final model
        y_pred, y_logit = nn.test(X_test)
        loss, _ = self.loss_function(y_logit, y_test) # softmax is included in entropy loss function
        acc = np.mean(y_pred == y_test)
        print('Last iteration - Test accuracy mean: {:.4f}, std: {:.4f}, loss: {:.4f}'.format(
            acc.mean(), acc.std(), loss))

In [ ]:
# Hyper-parameters
n_iter = 10000 # number of epochs
alpha = 1e-3 # learning_rate
mb_size = 64 # 2**10==1024 # width, timestep for sequential data or minibatch size
print_after = 10 # n_iter//10 # print loss for train, valid, and test
num_hidden_units = 32 # number of kernels/ filters in each layer
num_input_units = X_train.shape[1] # noise added at the input lavel as input noise we can use dX or for more improvement
num_output_units = y_train.max() + 1 # number of classes in this classification problem
num_layers = 1 # depth 

# Build the model/NN and learn it: running session.
nn = FFNN(C=num_output_units, D=num_input_units, H=num_hidden_units, L=num_layers)

nn.adam(train_set=(X_train, y_train), val_set=(X_val, y_val), mb_size=mb_size, alpha=alpha, 
           n_iter=n_iter, print_after=print_after)

Iter-10 train loss: 2.3797 valid loss: 2.4111, valid accuracy: 0.0730
Iter-20 train loss: 2.4200 valid loss: 2.4115, valid accuracy: 0.0734
Iter-30 train loss: 2.3824 valid loss: 2.4119, valid accuracy: 0.0740
Iter-40 train loss: 2.3652 valid loss: 2.4123, valid accuracy: 0.0748
Iter-50 train loss: 2.4052 valid loss: 2.4127, valid accuracy: 0.0770
Iter-60 train loss: 2.3905 valid loss: 2.4130, valid accuracy: 0.0776
Iter-70 train loss: 2.4025 valid loss: 2.4133, valid accuracy: 0.0790
Iter-80 train loss: 2.4223 valid loss: 2.4135, valid accuracy: 0.0804
Iter-90 train loss: 2.4195 valid loss: 2.4137, valid accuracy: 0.0818
Iter-100 train loss: 2.4054 valid loss: 2.4139, valid accuracy: 0.0824
Iter-110 train loss: 2.3928 valid loss: 2.4144, valid accuracy: 0.0826
Iter-120 train loss: 2.3707 valid loss: 2.4147, valid accuracy: 0.0828
Iter-130 train loss: 2.4502 valid loss: 2.4148, valid accuracy: 0.0838
Iter-140 train loss: 2.3837 valid loss: 2.4149, valid accuracy: 0.0842
Iter-150 train 

Iter-1170 train loss: 2.2981 valid loss: 2.3462, valid accuracy: 0.1370
Iter-1180 train loss: 2.3851 valid loss: 2.3439, valid accuracy: 0.1374
Iter-1190 train loss: 2.2988 valid loss: 2.3418, valid accuracy: 0.1374
Iter-1200 train loss: 2.3153 valid loss: 2.3397, valid accuracy: 0.1366
Iter-1210 train loss: 2.3075 valid loss: 2.3374, valid accuracy: 0.1360
Iter-1220 train loss: 2.3017 valid loss: 2.3350, valid accuracy: 0.1364
Iter-1230 train loss: 2.4119 valid loss: 2.3326, valid accuracy: 0.1376
Iter-1240 train loss: 2.2642 valid loss: 2.3303, valid accuracy: 0.1366
Iter-1250 train loss: 2.3812 valid loss: 2.3278, valid accuracy: 0.1370
Iter-1260 train loss: 2.3696 valid loss: 2.3254, valid accuracy: 0.1376
Iter-1270 train loss: 2.2464 valid loss: 2.3232, valid accuracy: 0.1380
Iter-1280 train loss: 2.3276 valid loss: 2.3207, valid accuracy: 0.1378
Iter-1290 train loss: 2.3216 valid loss: 2.3183, valid accuracy: 0.1376
Iter-1300 train loss: 2.3039 valid loss: 2.3160, valid accuracy:

Iter-2310 train loss: 1.9225 valid loss: 2.0164, valid accuracy: 0.3570
Iter-2320 train loss: 2.0296 valid loss: 2.0132, valid accuracy: 0.3592
Iter-2330 train loss: 1.9853 valid loss: 2.0099, valid accuracy: 0.3600
Iter-2340 train loss: 1.9651 valid loss: 2.0066, valid accuracy: 0.3602
Iter-2350 train loss: 2.0308 valid loss: 2.0033, valid accuracy: 0.3614
Iter-2360 train loss: 2.0204 valid loss: 2.0001, valid accuracy: 0.3622
Iter-2370 train loss: 2.0525 valid loss: 1.9969, valid accuracy: 0.3638
Iter-2380 train loss: 2.0134 valid loss: 1.9938, valid accuracy: 0.3644
Iter-2390 train loss: 2.0538 valid loss: 1.9905, valid accuracy: 0.3662
Iter-2400 train loss: 1.9894 valid loss: 1.9873, valid accuracy: 0.3670
Iter-2410 train loss: 1.9683 valid loss: 1.9839, valid accuracy: 0.3680
Iter-2420 train loss: 2.0767 valid loss: 1.9805, valid accuracy: 0.3712
Iter-2430 train loss: 2.0733 valid loss: 1.9772, valid accuracy: 0.3710
Iter-2440 train loss: 1.8906 valid loss: 1.9741, valid accuracy:

Iter-3450 train loss: 1.6402 valid loss: 1.6124, valid accuracy: 0.4298
Iter-3460 train loss: 1.7415 valid loss: 1.6082, valid accuracy: 0.4310
Iter-3470 train loss: 1.6257 valid loss: 1.6043, valid accuracy: 0.4322
Iter-3480 train loss: 1.5916 valid loss: 1.6003, valid accuracy: 0.4324
Iter-3490 train loss: 1.4407 valid loss: 1.5966, valid accuracy: 0.4328
Iter-3500 train loss: 1.5795 valid loss: 1.5929, valid accuracy: 0.4336
Iter-3510 train loss: 1.6008 valid loss: 1.5892, valid accuracy: 0.4346
Iter-3520 train loss: 1.5596 valid loss: 1.5851, valid accuracy: 0.4360
Iter-3530 train loss: 1.6803 valid loss: 1.5812, valid accuracy: 0.4374
Iter-3540 train loss: 1.6287 valid loss: 1.5772, valid accuracy: 0.4400
Iter-3550 train loss: 1.6087 valid loss: 1.5733, valid accuracy: 0.4412
Iter-3560 train loss: 1.4888 valid loss: 1.5696, valid accuracy: 0.4410
Iter-3570 train loss: 1.5229 valid loss: 1.5660, valid accuracy: 0.4420
Iter-3580 train loss: 1.6624 valid loss: 1.5620, valid accuracy:

Iter-4590 train loss: 1.2007 valid loss: 1.2288, valid accuracy: 0.6326
Iter-4600 train loss: 1.2119 valid loss: 1.2263, valid accuracy: 0.6344
Iter-4610 train loss: 1.2031 valid loss: 1.2237, valid accuracy: 0.6356
Iter-4620 train loss: 1.1286 valid loss: 1.2212, valid accuracy: 0.6370
Iter-4630 train loss: 1.3073 valid loss: 1.2183, valid accuracy: 0.6394
Iter-4640 train loss: 1.3215 valid loss: 1.2156, valid accuracy: 0.6426
Iter-4650 train loss: 1.2324 valid loss: 1.2129, valid accuracy: 0.6440
Iter-4660 train loss: 1.4172 valid loss: 1.2103, valid accuracy: 0.6460
Iter-4670 train loss: 1.3166 valid loss: 1.2077, valid accuracy: 0.6480
Iter-4680 train loss: 1.2544 valid loss: 1.2050, valid accuracy: 0.6496
Iter-4690 train loss: 1.2016 valid loss: 1.2024, valid accuracy: 0.6502
Iter-4700 train loss: 1.3886 valid loss: 1.1997, valid accuracy: 0.6514
Iter-4710 train loss: 1.1710 valid loss: 1.1972, valid accuracy: 0.6528
Iter-4720 train loss: 1.2528 valid loss: 1.1949, valid accuracy:

...

Iter-70180 train loss: 0.2900 valid loss: 0.3189, valid accuracy: 0.9064
Iter-70190 train loss: 0.1244 valid loss: 0.3189, valid accuracy: 0.9064
Iter-70200 train loss: 0.2225 valid loss: 0.3189, valid accuracy: 0.9066
Iter-70210 train loss: 0.4286 valid loss: 0.3189, valid accuracy: 0.9066
Iter-70220 train loss: 0.5835 valid loss: 0.3190, valid accuracy: 0.9068
Iter-70230 train loss: 0.4903 valid loss: 0.3190, valid accuracy: 0.9068
Iter-70240 train loss: 0.2592 valid loss: 0.3189, valid accuracy: 0.9068
Iter-70250 train loss: 0.2100 valid loss: 0.3189, valid accuracy: 0.9066
Iter-70260 train loss: 0.5272 valid loss: 0.3189, valid accuracy: 0.9068
Iter-70270 train loss: 0.2910 valid loss: 0.3189, valid accuracy: 0.9068
Iter-70280 train loss: 0.3326 valid loss: 0.3189, valid accuracy: 0.9068
Iter-70290 train loss: 0.2951 valid loss: 0.3188, valid accuracy: 0.9064
Iter-70300 train loss: 0.3734 valid loss: 0.3189, valid accuracy: 0.9062
Iter-70310 train loss: 0.5070 valid loss: 0.3189, v

Iter-71310 train loss: 0.3879 valid loss: 0.3179, valid accuracy: 0.9066
Iter-71320 train loss: 0.5473 valid loss: 0.3179, valid accuracy: 0.9068
Iter-71330 train loss: 0.3651 valid loss: 0.3178, valid accuracy: 0.9068
Iter-71340 train loss: 0.3149 valid loss: 0.3178, valid accuracy: 0.9070
Iter-71350 train loss: 0.3446 valid loss: 0.3178, valid accuracy: 0.9068
Iter-71360 train loss: 0.2087 valid loss: 0.3178, valid accuracy: 0.9066
Iter-71370 train loss: 0.6287 valid loss: 0.3178, valid accuracy: 0.9070
Iter-71380 train loss: 0.3971 valid loss: 0.3178, valid accuracy: 0.9066
Iter-71390 train loss: 0.1829 valid loss: 0.3178, valid accuracy: 0.9070
Iter-71400 train loss: 0.1823 valid loss: 0.3178, valid accuracy: 0.9070
Iter-71410 train loss: 0.3053 valid loss: 0.3178, valid accuracy: 0.9070
Iter-71420 train loss: 0.3471 valid loss: 0.3178, valid accuracy: 0.9068
Iter-71430 train loss: 0.2644 valid loss: 0.3178, valid accuracy: 0.9070
Iter-71440 train loss: 0.4114 valid loss: 0.3178, v

Iter-72440 train loss: 0.1705 valid loss: 0.3168, valid accuracy: 0.9080
Iter-72450 train loss: 0.1725 valid loss: 0.3168, valid accuracy: 0.9074
Iter-72460 train loss: 0.3207 valid loss: 0.3168, valid accuracy: 0.9074
Iter-72470 train loss: 0.1663 valid loss: 0.3167, valid accuracy: 0.9070
Iter-72480 train loss: 0.3985 valid loss: 0.3167, valid accuracy: 0.9072
Iter-72490 train loss: 0.2222 valid loss: 0.3167, valid accuracy: 0.9072
Iter-72500 train loss: 0.3632 valid loss: 0.3166, valid accuracy: 0.9072
Iter-72510 train loss: 0.2878 valid loss: 0.3166, valid accuracy: 0.9074
Iter-72520 train loss: 0.4564 valid loss: 0.3166, valid accuracy: 0.9076
Iter-72530 train loss: 0.3045 valid loss: 0.3166, valid accuracy: 0.9076
Iter-72540 train loss: 0.4854 valid loss: 0.3166, valid accuracy: 0.9078
Iter-72550 train loss: 0.2944 valid loss: 0.3165, valid accuracy: 0.9080
Iter-72560 train loss: 0.3255 valid loss: 0.3166, valid accuracy: 0.9078
Iter-72570 train loss: 0.3281 valid loss: 0.3166, v

Iter-73570 train loss: 0.4914 valid loss: 0.3161, valid accuracy: 0.9074
Iter-73580 train loss: 0.2864 valid loss: 0.3161, valid accuracy: 0.9074
Iter-73590 train loss: 0.3945 valid loss: 0.3161, valid accuracy: 0.9074
Iter-73600 train loss: 0.3017 valid loss: 0.3161, valid accuracy: 0.9076
Iter-73610 train loss: 0.3989 valid loss: 0.3161, valid accuracy: 0.9072
Iter-73620 train loss: 0.1541 valid loss: 0.3161, valid accuracy: 0.9072
Iter-73630 train loss: 0.5258 valid loss: 0.3161, valid accuracy: 0.9074
Iter-73640 train loss: 0.3297 valid loss: 0.3161, valid accuracy: 0.9072
Iter-73650 train loss: 0.1435 valid loss: 0.3161, valid accuracy: 0.9074
Iter-73660 train loss: 0.2407 valid loss: 0.3161, valid accuracy: 0.9072
Iter-73670 train loss: 0.2288 valid loss: 0.3161, valid accuracy: 0.9076
Iter-73680 train loss: 0.3902 valid loss: 0.3161, valid accuracy: 0.9078
Iter-73690 train loss: 0.2820 valid loss: 0.3161, valid accuracy: 0.9076
Iter-73700 train loss: 0.2389 valid loss: 0.3161, v

Iter-74700 train loss: 0.4090 valid loss: 0.3150, valid accuracy: 0.9078
Iter-74710 train loss: 0.3664 valid loss: 0.3149, valid accuracy: 0.9076
Iter-74720 train loss: 0.2681 valid loss: 0.3149, valid accuracy: 0.9076
Iter-74730 train loss: 0.3568 valid loss: 0.3149, valid accuracy: 0.9078
Iter-74740 train loss: 0.6379 valid loss: 0.3149, valid accuracy: 0.9078
Iter-74750 train loss: 0.2979 valid loss: 0.3149, valid accuracy: 0.9078
Iter-74760 train loss: 0.2754 valid loss: 0.3149, valid accuracy: 0.9074
Iter-74770 train loss: 0.5175 valid loss: 0.3149, valid accuracy: 0.9074
Iter-74780 train loss: 0.3478 valid loss: 0.3149, valid accuracy: 0.9074
Iter-74790 train loss: 0.3124 valid loss: 0.3149, valid accuracy: 0.9078
Iter-74800 train loss: 0.2905 valid loss: 0.3148, valid accuracy: 0.9080
Iter-74810 train loss: 0.3470 valid loss: 0.3148, valid accuracy: 0.9078
Iter-74820 train loss: 0.4916 valid loss: 0.3149, valid accuracy: 0.9076
Iter-74830 train loss: 0.4015 valid loss: 0.3149, v

In [ ]:
# # Display the learning curve and losses for training, validation, and testing
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

plt.plot(nn.losses['train'], label='Train loss')
# plt.plot(nn.losses['smooth train'], label='Train smooth loss')
plt.plot(nn.losses['valid'], label='Valid loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(nn.losses['valid_acc'], label='Valid accuracy')
plt.legend()
plt.show()